# Póker Mind - Análisis de Datos

**Introducción**

Este módulo del proyecto Póker Mind tiene como objetivo analizar en profundidad los datos generados previamente en nuestro dataset de simulaciones de póker. A través de un análisis exploratorio de datos (EDA), buscamos descubrir patrones, tendencias y relaciones clave que nos permitan entender mejor las probabilidades y los factores que influyen en el éxito de las manos de póker en diferentes escenarios.

**Objetivos del Análisis Exploratorio**

El análisis exploratorio de datos (EDA) en este módulo se organiza en una serie de preguntas clave que guían nuestra investigación. Estas preguntas están diseñadas para aprovechar al máximo la estructura de nuestro dataset y obtener respuestas a problemas fundamentales en el póker.

**Visualización del Dataset**

El dataset generado previamente incluye las siguientes columnas principales:

- `cartas_jugador`: Las dos cartas iniciales del jugador.
- `num_rivales`: Número de rivales en la partida simulada.
- `mano_preflop`: Clasificación de las cartas iniciales (por ejemplo, "Pareja" o "Carta Alta Suited").
- `flop`, `turn`, `river`: Cartas comunitarias en cada fase del juego.
- `mano_flop`, `mano_turn`, `mano_river`: Clasificación de la mano del jugador en cada etapa.
- `resultado`: Resultado final del juego (Victoria, Derrota, Empate).

Este conjunto de datos es ideal para responder preguntas estratégicas relacionadas con el póker y para identificar los factores que determinan el éxito en este juego. A lo largo del notebook, utilizaremos estas columnas para explorar y visualizar relaciones que nos ayuden a tomar decisiones más informadas en el póker.


##### **¡¡Observación!!**

Ten en cuenta que los porcentajes y datos que doy en los análisis de cada pregunta a la que respondo son estrictamente sacados de mi dataset actual, si tú deseas actualizar el dataset y/o re-ejecutar el notebook de generación de dataset para generar una simulación diferente de 2 millones de manos, tienes que sabes que los porcentajes que veas en las estadisticas no coincidirán al 100% con los que doy yo en el análisis, pero sí que deberían ser muy similares. Por lo tanto, es una buena practica incluso volver a ejecutar el notebook de generación del dataset para asegurarse que las estadísticas y los datos generados tienen coherencia y están funcionando corréctamente. Recuerda que cuantas más manos simuladas generes, más exactitud y veracidad tendrán las estadísticas generadas.

Dicho esto, disfruta del análisis del dataset.

### Importación de librerías

In [1]:
import pandas as pd
import plotly.express as px

### Carga y visualización del dataset

In [2]:
file_path = "../data/simulacion_montecarlo.csv"
df = pd.read_csv(file_path)

df.head()

,cartas_jugador,num_rivales,mano_preflop,flop,mano_flop,cartas_flop,turn,mano_turn,cartas_turn,river,mano_river,cartas_river,resultado
0,"['A♣', '4♥']",1,Carta Alta Offsuit,"['Q♥', 'J♠', '10♠']",Carta Alta,"['A♣', '4♥', 'Q♥', 'J♠', '10♠']",['9♣'],Carta Alta,"['A♣', 'Q♥', 'J♠', '10♠', '9♣']",['9♦'],Pareja,"['A♣', 'Q♥', 'J♠', '9♣', '9♦']",Derrota
1,"['7♦', '10♣']",3,Carta Alta Offsuit,"['5♥', '6♣', '7♠']",Pareja,"['7♦', '10♣', '5♥', '6♣', '7♠']",['9♥'],Pareja,"['7♦', '10♣', '6♣', '7♠', '9♥']",['8♣'],Escalera,"['7♦', '10♣', '6♣', '9♥', '8♣']",Victoria
2,"['8♣', '7♦']",7,Carta Alta Offsuit,"['9♦', '7♠', '10♥']",Pareja,"['8♣', '7♦', '9♦', '7♠', '10♥']",['Q♠'],Pareja,"['7♦', '9♦', '7♠', '10♥', 'Q♠']",['5♦'],Pareja,"['7♦', '9♦', '7♠', '10♥', 'Q♠']",Derrota
3,"['K♥', '9♥']",1,Carta Alta Suited,"['10♠', '6♥', 'K♣']",Pareja,"['K♥', '9♥', '10♠', '6♥', 'K♣']",['4♦'],Pareja,"['K♥', '9♥', '10♠', '6♥', 'K♣']",['7♠'],Pareja,"['K♥', '9♥', '10♠', 'K♣', '7♠']",Victoria
4,"['4♣', 'J♦']",7,Carta Alta Offsuit,"['5♥', 'J♠', 'Q♠']",Pareja,"['4♣', 'J♦', '5♥', 'J♠', 'Q♠']",['3♦'],Pareja,"['4♣', 'J♦', '5♥', 'J♠', 'Q♠']",['5♣'],Doble Pareja,"['J♦', '5♥', 'J♠', 'Q♠', '5♣']",Derrota


## EDA

### 1. **Manos iniciales y probabilidades de éxito**

#### 1.1 ¿Cuáles son las manos iniciales (preflop) con mayor probabilidad de victoria?

In [3]:
# Calculamos la tasa de victoria por tipo de mano preflop
victory_rates = (
    df.groupby('mano_preflop')['resultado']
    .apply(lambda x: (x == 'Victoria').mean())
    .reset_index()
)
victory_rates.columns = ['Tipo de Mano Preflop', 'Tasa de Victoria']

# Grafico de barras con Plotly
fig = px.bar(
    victory_rates,
    x='Tipo de Mano Preflop',
    y='Tasa de Victoria',
    color='Tipo de Mano Preflop',  
    text='Tasa de Victoria', 
    title="Tasa de Victoria por Tipo de Mano Preflop",
    labels={"Tipo de Mano Preflop": "Tipo de Mano", "Tasa de Victoria": "Tasa de Victoria (%)"},
    color_discrete_sequence=px.colors.qualitative.Vivid  
)

# Formato del gráfico
fig.update_traces(texttemplate='%{text:.2%}', textposition='outside') 
fig.update_layout(
    xaxis=dict(
        title="Tipo de Mano Preflop",
        title_font=dict(size=18),
        tickfont=dict(size=14),
    ),
    yaxis=dict(
        title="Tasa de Victoria",
        title_font=dict(size=18),
        tickfont=dict(size=14),
    ),
    title=dict(
        font=dict(size=20),
        x=0.5,  
    ),
    margin=dict(l=40, r=40, t=60, b=40),  
    plot_bgcolor="White"  
)

fig.show()

##### **Análisis de la Tasa de Victoria por Tipo de Mano Preflop**

La gráfica muestra que las **parejas** tienen la mayor tasa de victoria preflop con un 31.66%. Le siguen las **cartas altas suited** (23.75%) y las **cartas altas offsuit** (20.00%). Las parejas tienen una clara ventaja sobre los otros tipos de mano.

##### **Conclusión**

Las **parejas** son las manos preflop más fuertes, seguidas de las **cartas altas suited**. Las **cartas altas offsuit** tienen la menor probabilidad de victoria, pero aún pueden ser competitivas según las cartas comunitarias.


#### 1.2 ¿Qué porcentaje de Parejas Preflop conectan un Trío en cada etapa de la mano (flop, turn y river)?

In [4]:
# Filtramos las manos preflop que son parejas
pair_hands = df[df['mano_preflop'] == 'Pareja Offsuit'].copy()

# Calculamos la probabilidad de conectar un trío en cada etapa
pair_hands['trio_flop'] = pair_hands['mano_flop'].str.contains('Trío')
pair_hands['trio_turn'] = pair_hands['mano_turn'].str.contains('Trío') & ~pair_hands['trio_flop']
pair_hands['trio_river'] = pair_hands['mano_river'].str.contains('Trío') & ~pair_hands['trio_flop'] & ~pair_hands['trio_turn']

# Calculamos porcentajes de probabilidad
flop_rate = pair_hands['trio_flop'].mean() * 100
turn_rate = pair_hands['trio_turn'].mean() * 100
river_rate = pair_hands['trio_river'].mean() * 100
total_rate = pair_hands[['trio_flop', 'trio_turn', 'trio_river']].any(axis=1).mean() * 100

# Creamos un dataframe para la visualización
stages = ['Flop', 'Turn', 'River', 'Total']
rates = [flop_rate, turn_rate, river_rate, total_rate]
probabilities = pd.DataFrame({'Etapa': stages, 'Probabilidad (%)': rates})

In [5]:
# Visualización
fig = px.line(
    probabilities,  
    x='Etapa',     
    y='Probabilidad (%)',
    markers=True,   
    title="Probabilidad de Conectar un Trío con Pareja Preflop por Etapa",  
    labels={
        "Etapa": "Etapa de la Mano", 
        "Probabilidad (%)": "Probabilidad (%)"
    },
    color_discrete_sequence=['royalblue'] 
)

# Ajustes adicionales al diseño
fig.update_traces(
    line=dict(width=2.5), 
    marker=dict(size=9),  
)

fig.update_layout(
    title=dict(
        font=dict(size=20, family="Arial"),  
        x=0.5  
    ),
    xaxis=dict(
        title="Etapa de la Mano", 
        title_font=dict(size=16),  
        tickfont=dict(size=12),    
    ),
    yaxis=dict(
        title="Probabilidad (%)",  
        title_font=dict(size=16), 
        tickfont=dict(size=12),    
        range=[0, probabilities['Probabilidad (%)'].max() + 5] 
    ),
    plot_bgcolor="White", 
    margin=dict(l=40, r=40, t=60, b=40),  
)

# Anotaciones con los porcentajes
for i, row in probabilities.iterrows():
    fig.add_annotation(
        x=row['Etapa'],
        y=row['Probabilidad (%)'] + 2,
        text=f"{row['Probabilidad (%)']:.2f}%",
        showarrow=False,
        arrowhead=2,    
        ax=0, ay=-30,   
        font=dict(size=12, color="black"), 
        arrowcolor="black",  
        bgcolor="white",     
        opacity=0.8         
    )

fig.show()

##### **Análisis de la probabilidad de conectar un trío**

Las parejas preflop tienen una probabilidad acumulada del 16.14% de conectar un trío en alguna etapa de la mano. Este resultado se distribuye de la siguiente manera:

- Flop: La mayor probabilidad de conectar ocurre aquí (10.84%) porque tres cartas comunitarias se revelan de golpe.
- Turn: Si no se conectó en el flop, la probabilidad de conectar en esta etapa baja al 2.94%.
- River: La probabilidad es aún menor (2.36%) para conectar con la última carta.

En conjunto, este análisis destaca la importancia del flop para mejorar la mano con una pareja inicial. Esto también subraya la dificultad de formar tríos en las etapas posteriores, reforzando que en póker, cada etapa presenta oportunidades decrecientes para mejorar una mano.

#### 1.3 ¿Cuál es la probabilidad de que los Suited Connectors conecten Escalera, Color o Escalera de Color?

En póker, los *Suited Connectors* (dos cartas consecutivas del mismo palo) son manos con gran potencial para formar combinaciones fuertes como escaleras, colores o incluso escaleras de color. En este análisis, evaluaremos qué porcentaje de estas manos preflop terminan conectando alguno de estos proyectos al final de la mano.

In [6]:
# Función para filtrar manos preflop que son Suited Connectors
def is_suited_connector(cards):
    ranks = {'2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, 'J': 11, 'Q': 12, 'K': 13, 'A': 14}
    card1, card2 = cards
    return (
        card1[-1] == card2[-1] and  # Mismo palo
        abs(ranks[card1[:-1]] - ranks[card2[:-1]]) == 1  # Consecutivas
    )

In [7]:
# Extraemos los suited connectors del dataset
df['suited_connector'] = df['cartas_jugador'].apply(eval).apply(is_suited_connector)
suited_connectors = df[df['suited_connector']].copy()

# Evaluamos si conectan proyectos
suited_connectors.loc[:, 'escalera'] = suited_connectors['mano_river'].str.contains('Escalera')
suited_connectors.loc[:, 'color'] = suited_connectors['mano_river'].str.contains('Color')
suited_connectors.loc[:, 'escalera_color'] = suited_connectors['mano_river'].str.contains('Escalera de Color')

# Calculamos % de probabilidad de conectar cada proyecto
escalera_rate = suited_connectors['escalera'].mean() * 100
color_rate = suited_connectors['color'].mean() * 100
escalera_color_rate = suited_connectors['escalera_color'].mean() * 100
total_rate = suited_connectors[['escalera', 'color', 'escalera_color']].any(axis=1).mean() * 100

# Creamos un dataframe para la visualización
categories = ['Escalera', 'Color', 'Escalera de Color', 'Cualquier Proyecto']
rates = [escalera_rate, color_rate, escalera_color_rate, total_rate]
results = pd.DataFrame({'Categoría': categories, 'Probabilidad (%)': rates})

In [8]:
# Visualización
fig = px.bar(
    results,
    x='Categoría',
    y='Probabilidad (%)',
    text='Probabilidad (%)',
    title="Probabilidad de Conectar Proyectos con Suited Connectors",
    labels={
        "Categoría": "Tipo de Proyecto",
        "Probabilidad (%)": "Probabilidad (%)"
    },
    color='Probabilidad (%)',
    color_continuous_scale='Reds'  
)

# Ajustes adicionales al diseño
fig.update_traces(
    texttemplate='%{text:.2f}%', 
    textposition='outside',     
    marker=dict(line=dict(color='black', width=0.5)) 
)

fig.update_layout(
    title=dict(
        font=dict(size=20, family="Arial"),  
        x=0.5  
    ),
    xaxis=dict(
        title="Categoría", 
        title_font=dict(size=16), 
        tickfont=dict(size=12),   
    ),
    yaxis=dict(
        title="Probabilidad (%)",  
        title_font=dict(size=16), 
        tickfont=dict(size=12),    
        range=[0, max(rates) + 5]  
    ),
    plot_bgcolor="White", 
    margin=dict(l=40, r=40, t=60, b=40), 
    coloraxis_showscale=True 
)

fig.show()

##### **Análisis de la probabilidad de conectar proyectos con *Suited Connectors***

Hemos observado en esta gráfica que los *Suited Connectors* son manos muy interesantes de jugar ya que en partidas de 2-9 jugadores conectan proyectos el 13.73% divididos en:

- Escalera: 7.22%
- Color: 6.65%
- Escalera de color: 0.13%

Esto sin tener en cuenta proyectos que pueden llegar a conectar de Dobles Parejas, Tríos e incluso Full Houses en algunos de los casos.

#### 1.4 ¿Cuál es y cómo varía el porcentaje de victorias preflop según la fuerza de la mano y sus características? ¿Cuál es la mano más fuerte? ¿Y la más débil?

Para realizar este análisis y poder visualizarlo mediante un Scatterplot o nube de puntos, lo primero que tenemos que hacer es guardar en un dataframe cada combinación de dos cartas que te pueden salir del mazo de Poker. Cómo el dataset no sabe que [A♠, A♦] y [A♦, A♠] es lo mismo, tenemos que normalizar las manos que sean iguales pero en distinto orden y quedarnos con la media de las proporciones de victoria de ambas manos.

Obtendremos un total de 1326 manos posibles.

In [9]:
# Normalizamos las manos para eliminar duplicados
def normalize_hand(hand):
    cards = eval(hand)
    return str(sorted(cards))

df['normalized_hand'] = df['cartas_jugador'].apply(normalize_hand)

Seguidamente tendremos que calcular el porcentaje de victoria que tiene cada una de las 1326 manos y calcular la probabilidad que tiene el jugador de ganar una partida antes de repartir cartas en manos de 2-9 jugadores. Para ello calculamos la media de jugadores que hay en el millón de simulaciones que tenemos como referencia y sacamos la probabilidad de ganar contra ese número de jugadores. Lo utilizaremos como recta de regresión en nuestro Scatterplot.

In [10]:
# Calculamos el porcentaje de victoria para manos únicas
df['resultado_bin'] = (df['resultado'] == 'Victoria').astype(int)
victory_rate_by_hand = df.groupby('normalized_hand')['resultado_bin'].mean() * 100

# Calculamos la probabilidad media esperada
df['expected_win_rate'] = 100 / (df['num_rivales'] + 1)

Por último, visualizamos los datos en un Scatterplot con leyenda para dividir cada tipo de mano en colores, información de cada mano específica al hacer hover sobre ella, recta de regresión lineal para tener como referencia la probabilidad media de ganar una mano etc.

In [11]:
# Creamos una métrica para el eje X: fuerza de la mano
card_values = {str(i): i for i in range(2, 10)}
card_values.update({'10': 10, 'J': 11, 'Q': 12, 'K': 13, 'A': 14})

def hand_strength(hand):
    cards = eval(hand)
    return sum(card_values[card[:-1]] for card in cards)

scatter_data = victory_rate_by_hand.reset_index()
scatter_data.columns = ['Cartas', 'Win Rate']
scatter_data['Strength'] = scatter_data['Cartas'].apply(hand_strength)

# Categorizamos las manos para colorear el scatterplot
def categorize_hand(hand):
    cards = eval(hand)
    values = [card[:-1] for card in cards]
    suits = [card[-1] for card in cards]
    value_ranks = sorted([card_values[v] for v in values])
    
    if values[0] == values[1]:
        return "Parejas"
    elif suits[0] == suits[1]:
        if abs(value_ranks[1] - value_ranks[0]) == 1:
            return "Conectores Suited"
        else:
            return "Cartas Distintas Suited"
    else:
        if abs(value_ranks[1] - value_ranks[0]) == 1:
            return "Conectores Offsuit"
        else:
            return "Cartas Distintas Offsuit"

# Guardamos la categoría en el dataframe
scatter_data['Category'] = scatter_data['Cartas'].apply(categorize_hand)

# Creamos el scatterplot con la categoría como color
fig = px.scatter(
    scatter_data,
    x='Strength',
    y='Win Rate',
    color='Category',
    hover_data={
        'Cartas': True,
        'Win Rate': ':.2f',
    },
    title="Porcentaje de Victoria de Manos Preflop Exactas",
    labels={"Strength": "Fuerza de la Mano", "Win Rate": "Porcentaje de Victoria", "Category": "Categoría"},
    color_discrete_map={
        "Parejas": "blue",
        "Cartas Distintas Offsuit": "orange",
        "Cartas Distintas Suited": "green",
        "Conectores Suited": "purple",
        "Conectores Offsuit": "red",
    }
)

# Agregamos la recta de regresión lineal como referencia
average_expected_rate = df['expected_win_rate'].mean()
fig.add_shape(
    type="line",
    x0=scatter_data['Strength'].min(),
    y0=average_expected_rate,
    x1=scatter_data['Strength'].max(),
    y1=average_expected_rate,
    line=dict(color="Red", width=2, dash="dash"),
    xref="x",
    yref="y"
)

# Añadimos una anotación con la probabilidad media esperada
fig.add_annotation(
    x=scatter_data['Strength'].mean(),  
    y=average_expected_rate * 2,
    xref="x",
    yref="y",
    text=f"Probabilidad media esperada ({average_expected_rate:.2f}%)",
    showarrow=False,
    font=dict(color="Red", size=14),
    align="center",
    bgcolor="white",
    bordercolor="red",
    borderwidth=1,
    opacity=0.8
)

# Ajustamos el diseño del gráfico
fig.update_traces(marker=dict(size=8, opacity=0.8, line=dict(width=1, color='DarkSlateGrey')))
fig.update_layout(
    legend=dict(title="Categoría de Mano"),
    title=dict(font=dict(size=20, family="Arial"), x=0.5),
    margin=dict(l=40, r=40, t=60, b=40)
)

# Mostrar el gráfico
fig.show()

##### **Análisis del porcentaje de victoria de cada mano**

Gracias a este Scatterplot estamos viendo a primer golpe de vista que la mano con menos proporción de victoria es el [2, 6] Offsuit. En el caso de nuestro dataset (recuerda que este porcentaje exacto varía si vuelves a generar el dataset con 2 millones de manos simuladas) el porcentaje de victoria de esta mano es del 10.24% mientras que, evidentemente, la mano más fuerte [A, A] tiene una tasa de victoria de en torno al 55% cada combinación de manos [A, A]

También se puede apreciar muy rapido que en cada nivel de fuerza de la mano (la medimos por la suma de los números de las dos cartas) las parejas predominan en todos los niveles de fuerza par, ya que en el impar es imposible formar una pareja. Sin embargo, en todos los niveles vemos que las peores manos siempre están representadas por las cartas distintas suited.

En resumen, si te tiran All In preflop, ten muy en cuenta esta estadística para plantearte ver el All In estríctamente si la mano que posees tiene un % de victoria por encima de la recta de regresión.

### 2. **Tipos de manos finales**

#### 2.1 ¿Cuál es la proporción de cada tipo de mano que se forma al final de una mano de Poker?

In [12]:
# Calculamos la proporción de cada tipo de mano final
hand_type_distribution = df['mano_river'].value_counts(normalize=True) * 100
hand_type_distribution = hand_type_distribution.reset_index()
hand_type_distribution.columns = ['Tipo de Mano', 'Proporción (%)']

In [13]:
# Visualización en un gráfico de rosquilla
fig = px.pie(
    hand_type_distribution,
    names='Tipo de Mano',
    values='Proporción (%)',
    title='Proporción de Cada Tipo de Mano Final',
    hole=0.4,
    color='Tipo de Mano',
    color_discrete_sequence=px.colors.qualitative.Set2
)

# Diseño
fig.update_layout(
    title=dict(font=dict(size=20, family="Arial"), x=0.5),
    annotations=[
        dict(
            text=f"Total: {df.shape[0]:,} manos",
            x=0.5, y=1.1,
            font=dict(size=15, family="Arial"),
            showarrow=False
        )
    ]
)

fig.show()

##### **Análisis de las proporciones de cada tipo de mano final**

En este gráfico de Rosquilla, hemos observado que el tipo de mano mas frecuente con diferencia es la *Pareja*. Esto tiene mucho sentido ya que para conseguir este tipo de mano sólo tienes que conectar 2 cartas iguales entre las 7 posibles cartas de la mano.

A este tipo de mano le siguen las *Dobles Parejas* y la *Carta Alta* con porcentajes bastante similares. Es sorprendente ver que sean más frecuenten las *Dobles Parejas* que la *Carta Alta* pero tiene todo el sentido del mundo.

Finalmente vemos manos como *Trío* y *Escalera* que son muy parecidas porcentualmente. A estas manos las siguen también muy similares, el *Color* y el *Full House* con porcentajes bastante similares, y por último tenemos las tres manos menos probables que son *Poker*, *Escalera de Color* y *Escalera Real* con porcentajes tan bajos que no suman ni un 0.25% entre las 3. Son manos realmente poco comunes.

#### 2.2 ¿Cuál es la probabilidad de victoria asociada con cada tipo de mano final?

Una vez analizada la rareza de cada mano, queremos ver cómo de probable es ser la mano mas fuerte de la partida con cada uno de los tipos de manos. posibles

In [14]:
# Calculamos la proporción de victorias por tipo de mano final y convertimos a porcentaje
victory_rate_by_hand_type = (
    df.groupby('mano_river')['resultado_bin']
    .mean()
    .reset_index()
    .rename(columns={'resultado_bin': 'Win Rate'})
    .sort_values(by='Win Rate', ascending=False)
)

victory_rate_by_hand_type['Win Rate'] *= 100

In [15]:
# Visualización en un gráfico de barras horizontales
fig = px.bar(
    victory_rate_by_hand_type,
    x='Win Rate',
    y='mano_river',
    orientation='h',
    text='Win Rate',
    title="Proporción de Victoria Según el Tipo de Mano Final",
    labels={"mano_river": "Tipo de Mano Final", "Win Rate": "Proporción de Victoria (%)"},
    color='Win Rate',
    color_continuous_scale=px.colors.sequential.Viridis,
)

# Diseño de las barras
fig.update_traces(
    texttemplate='%{text:.2f}%', 
    textposition='outside',    
    marker=dict(
        line=dict(color='black', width=0.5),  
        opacity=0.85                          
    )
)

# Diseño general
fig.update_layout(
    title=dict(
        text="Proporción de Victoria Según el Tipo de Mano Final",
        font=dict(size=24, family="Arial", color="Black"),
        x=0.5
    ),
    xaxis=dict(
        title="Proporción de Victoria (%)",
        title_font=dict(size=18, family="Arial", color="Gray"),
        tickfont=dict(size=14, family="Arial", color="Gray"),
        gridcolor="LightGray",  
        zeroline=False         
    ),
    yaxis=dict(
        title="Tipo de Mano Final",
        title_font=dict(size=18, family="Arial", color="Gray"),
        tickfont=dict(size=14, family="Arial", color="Gray"),
    ),
    coloraxis_showscale=False,  
    margin=dict(l=100, r=50, t=80, b=50),  
    plot_bgcolor="White",
)

fig.show()

##### **Análisis de la proporción de victoria según el tipo de mano final**

En este gráfico podemos observar que efectivamente las manos menos comunes que habíamos visto en el grafico anterior, *Poker, Escalera de Color y Escalera Real*, son manos realmente fuertes y la tasa de victoria es altísima.

De hecho este análisis lo realizamos para todas las manos en el River únicamente, es por eso que el *Poker* por ejemplo tiene un 13.14% en el que pierde o empata, porque habrá alguna simulación en la que nos encontremos el *Poker* en las cartas comunitarias y habrá empates y partidas decididas por el *Kicker*. Este tipo de manos, si empezamos con proyecto preflop tienen una tasa de victoria que tiende a 100.

También podemos observar que la tasa de *Escalera Real* es prácticamente 100%. Es una mano que sale aproximadamente 1 de cada 30,000 veces pero esa vez que es prácticamente victoria asegurada.

Por último, es muy destacable el bajo porcentaje de victoria de *Carta Alta*, en nuestro caso un 2.51%. Esto nos da una idea de que en la gran mayoría de los casos si nos vamos al River con *Carta Alta* es una mano que ganaremos 1 de cada 40 veces aproximádamente en partidas de 2-9 jugadores, unos 4-5 jugadores de media.